<a href="https://colab.research.google.com/github/go-hyun77/ABSA/blob/fixed-incorrect-labels-in-dataset-preprocessing-and-retrain/ABSA_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Aspect-Based Sentiment Analysis (ABSA) with T5
# --------------------------------------------------
# This notebook shows how to fine-tune a T5 model for ABSA using HuggingFace.
# SemEval2014 dataset (aspect + sentiment annotations).

!pip install transformers datasets sentencepiece -q
!pip install datasets==3.6.0

import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Load Dataset

dataset = load_dataset("alexcadillon/SemEval2014Task4", "restaurants")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
# examine dataset
train_data = dataset["train"]

# print first 10 entries of train split
for i in range(10):
    print(f"{i+1}: {train_data[i]}")


1: {'sentenceId': '3121', 'text': 'But the staff was so horrible to us.', 'aspectTerms': [{'term': 'staff', 'polarity': 'negative', 'from': '8', 'to': '13'}], 'aspectCategories': [{'category': 'service', 'polarity': 'negative'}]}
2: {'sentenceId': '2777', 'text': "To be completely fair, the only redeeming factor was the food, which was above average, but couldn't make up for all the other deficiencies of Teodora.", 'aspectTerms': [{'term': 'food', 'polarity': 'positive', 'from': '57', 'to': '61'}], 'aspectCategories': [{'category': 'food', 'polarity': 'positive'}, {'category': 'anecdotes/miscellaneous', 'polarity': 'negative'}]}
3: {'sentenceId': '1634', 'text': "The food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel like eating, whether it's on the menu or not.", 'aspectTerms': [{'term': 'food', 'polarity': 'positive', 'from': '4', 'to': '8'}, {'term': 'kitchen', 'polarity': 'positive', 'from': '55', 'to': '62'}, {'term': 'menu', 'p

In [4]:
# flatten dataset
indexes = [train_data[i] for i in range(20)]  # first 20 entries


rows = []
for i in indexes:
    sentence_id = i["sentenceId"]
    text = i["text"]

    # If aspect terms exist, iterate through them
    if i["aspectTerms"]:
        for asp in i["aspectTerms"]:
            rows.append({
                "sentenceId": sentence_id,
                "text": text,
                "aspect_term": asp["term"],
                "term_polarity": asp["polarity"],
                "category": None,  # Add these to maintain consistent columns
                "category_polarity": None # Add these to maintain consistent columns
            })
    # If no explicit aspect terms, still record categories
    if i["aspectCategories"]:
        for cat in i["aspectCategories"]:
            rows.append({
                "sentenceId": sentence_id,
                "text": text,
                "aspect_term": None, # Add these to maintain consistent columns
                "term_polarity": None, # Add these to maintain consistent columns
                "category": cat["category"],
                "category_polarity": cat["polarity"]
            })


# Convert to DataFrame
df = pd.DataFrame(rows)
print(df.head(10))

  sentenceId                                               text aspect_term  \
0       3121               But the staff was so horrible to us.       staff   
1       3121               But the staff was so horrible to us.        None   
2       2777  To be completely fair, the only redeeming fact...        food   
3       2777  To be completely fair, the only redeeming fact...        None   
4       2777  To be completely fair, the only redeeming fact...        None   
5       1634  The food is uniformly exceptional, with a very...        food   
6       1634  The food is uniformly exceptional, with a very...     kitchen   
7       1634  The food is uniformly exceptional, with a very...        menu   
8       1634  The food is uniformly exceptional, with a very...        None   
9       2534  Where Gabriela personaly greets you and recomm...        None   

  term_polarity                 category category_polarity  
0      negative                     None              None  
1       

In [5]:
#define model

model_name = "t5-small" #try "google/flan-t5-base" for better results
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
#create aspect-sentiment pairs from dataset

def format_target(ex):
    pairs = []
    for asp in ex.get("aspectTerms", []):
        term = asp["term"]
        pol = asp["polarity"]
        pairs.append(f"{term}: {pol}")
    return "; ".join(pairs) if pairs else "no aspects"


In [7]:
#function to tokenize inputs (as in the plain sentences + aspect terms/values) for model to train on

def preprocess(ex):
    input_text = f"ABSA: {ex['text']}"
    target_text = format_target(ex)
    return tokenizer(
        input_text,
        text_target=target_text,
        truncation=True,
        padding="max_length",
        max_length=128
    )

In [8]:
#apply preprocess function to each entry in training and validation test splits
train_dataset = dataset["train"].map(preprocess)
valid_dataset = dataset["test"].map(preprocess)


In [9]:
print(train_dataset[0])

{'sentenceId': '3121', 'text': 'But the staff was so horrible to us.', 'aspectTerms': [{'term': 'staff', 'polarity': 'negative', 'from': '8', 'to': '13'}], 'aspectCategories': [{'category': 'service', 'polarity': 'negative'}], 'input_ids': [20798, 188, 10, 299, 8, 871, 47, 78, 17425, 12, 178, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [10]:
#load model
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [11]:
#training setup and parameters

args = TrainingArguments(
  output_dir="./absa_t5",
  eval_strategy="epoch", # Corrected parameter name
  learning_rate=5e-5,
  per_device_train_batch_size=8,
  num_train_epochs=5,
  weight_decay=0.01,
  save_total_limit=2,
  logging_steps=50,
  push_to_hub=False,
)


trainer = Trainer(
  model=model,
  args=args,
  train_dataset=train_dataset,
  eval_dataset=valid_dataset,
)

In [12]:
#train model, no need to execute this block if loading saved model
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gohyun (gohyun-california-state-university-fullerton) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,0.088400,0.041716
2,0.039300,0.028546
3,0.035500,0.025298
4,0.034200,0.024151
5,0.026000,0.023684


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=1905, training_loss=0.18660602854305677, metrics={'train_runtime': 24234.8924, 'train_samples_per_second': 0.627, 'train_steps_per_second': 0.079, 'total_flos': 514468022845440.0, 'train_loss': 0.18660602854305677, 'epoch': 5.0})

In [13]:
#mount drive folder for saving trained model
!fusermount -u /content/drive
!rm -rf /content/drive
from google.colab import drive
drive.mount('/content/drive')

!ls /content/drive/MyDrive

Mounted at /content/drive
'3rd Iteration Document'  'CPSC 301'	  'CPSC 439'  'CPSC 566'
 ABSA_T5_Model		  'CPSC 311'	  'CPSC 440'  'CPSC 585'
'AP GOV'		  'CPSC 315'	  'CPSC 452'  'CPSC 589'
 BIO101			  'CPSC 323'	  'CPSC 471'  'EGCP 401'
 Books			  'CPSC 332'	  'CPSC 481'  'EVO Food Places.xlsx'
'Colab Notebooks'	  'CPSC 335'	  'CPSC 485'   MATH338
'CPSC 121'		  'CPSC 351'	  'CPSC 531'   Misc.
'CPSC 223J'		  'CPSC 353 458'  'CPSC 544'  'Oct Genesis.png'
'CPSC 240'		  'CPSC 362'	  'CPSC 548'  'PSC Biotech'
'CPSC 254'		  'CPSC 375'	  'CPSC 552'  'Test Folder'


In [14]:
#save model

model_dir = "/content/drive/MyDrive/ABSA_T5_Model"
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)



('/content/drive/MyDrive/ABSA_T5_Model/tokenizer_config.json',
 '/content/drive/MyDrive/ABSA_T5_Model/special_tokens_map.json',
 '/content/drive/MyDrive/ABSA_T5_Model/spiece.model',
 '/content/drive/MyDrive/ABSA_T5_Model/added_tokens.json')

In [15]:
# Load tokenizer and model from your Drive
tokenizer = T5Tokenizer.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)


print("Model path:", model.config._name_or_path)
print("Number of parameters:", sum(p.numel() for p in model.parameters()) // 1e6, "M")

Model path: /content/drive/MyDrive/ABSA_T5_Model
Number of parameters: 60.0 M


In [16]:
#test model with text input

# 1️⃣ Confirm model path
print("Model path:", model.config._name_or_path)

# 2️⃣ Confirm the prefix was used during training
print("Example training input:", dataset["train"][0]["text"])

# 3️⃣ Try inference without prefix (if you didn't train with one)
def absa_predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    outputs = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(absa_predict("The food was amazing but the service was terrible."))


Model path: /content/drive/MyDrive/ABSA_T5_Model
Example training input: But the staff was so horrible to us.
food was positive; service was negative
